In [1]:
!pip install pyomo -q
!pip install mistralai -q
!wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


In [2]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from IPython.display import display, Markdown, Latex
from datetime import datetime

In [3]:
API_KEY = ''

MODEL_ID = 'open-mixtral-8x22b'
MODEL_SEED = 2
MODEL_TEMPERATURE = 0.7

SYSTEM_PROMPT_1 = """Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."""
SYSTEM_PROMPT_2 = """Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."""
PROBLEM_DESCRIPTION = """We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number.

We want to maximize the daily profit of this alkylation process.
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup.

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas.
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed.
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate.
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup."""


client = MistralClient(api_key=API_KEY)

print(f'Time of execution: {datetime.now()}')

Time of execution: 2024-06-14 15:06:56.602108


## Step 1 - Generate Mathematical Formulation

In [4]:
messages_1 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_1),
    ChatMessage(role="user", content=PROBLEM_DESCRIPTION)
]

In [5]:
response_1 = client.chat(
    model=MODEL_ID,
    messages=messages_1,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_1_text = response_1.choices[0].message.content

In [6]:
Markdown(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the parameters, decision variables, objective function, and constraints as follows:

Parameters:

* $C_{AF}$: Cost per barrel of olefin feed
* $C_{IR}$: Cost per barrel of isobutane recycle
* $C_{AAR}$: Cost per thousand pounds of acid addition rate
* $C_{IM}$: Cost per barrel of isobutane makeup
* $P_{AY}$: Price per barrel of alkylate yield
* $P_{MON}$: Price per unit of motor octane number

Decision Variables:

* $x_{1}$: Olefin feed (barrels per day)
* $x_{2}$: Isobutane recycle (barrels per day)
* $x_{3}$: Acid addition rate (thousands of pounds per day)
* $x_{4}$: Alkylate yield (barrels per day)
* $x_{5}$: Isobutane makeup (barrels per day)
* $x_{6}$: Acid strength (weight %)
* $x_{7}$: Motor octane number
* $x_{8}$: External isobutane-to-olefin ratio
* $x_{9}$: Acid dilution factor
* $x_{10}$: F-4 performance number

Objective Function:

The objective is to maximize the daily profit, which is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs. The objective function can be formulated as follows:

$$\max Z = P_{AY} \cdot x_{4} \cdot x_{7} - (C_{AF} \cdot x_{1} + C_{IR} \cdot x_{2} + C_{AAR} \cdot x_{3} + C_{IM} \cdot x_{5})$$

Constraints:

1. Alkylate Yield:

$$x_{4} = x_{1} \cdot (1.12 + 0.13167 \cdot x_{8} - 0.00667 \cdot x_{8}^2)$$

2. Motor Octane Number:

$$x_{7} = 86.35 + 1.098 \cdot x_{8} - 0.038 \cdot x_{8}^2 + 0.325 \cdot (x_{6} - 89)$$

3. Acid Dilution Factor:

$$x_{9} = 35.82 - 0.222 \cdot x_{10}$$

4. F-4 Performance Number:

$$x_{10} = -133 + 3 \cdot x_{7}$$

5. External Isobutane-to-Olefin Ratio:

$$x_{8} = \frac{x_{2} + x_{5}}{x_{1}}$$

6. Acid Strength:

$$x_{6} = \frac{93000 \cdot x_{3}}{x_{3} \cdot x_{9} + 1000 \cdot x_{3}}$$

7. Alkylate Yield Constraint:

$$1.22 \cdot x_{4} = x_{1} + x_{5}$$

8. Lower and Upper Bounds:

Each decision variable $x_{i}$ has a lower bound $LB_{i}$ and an upper bound $UB_{i}$:

$$LB_{i} \leq x_{i} \leq UB_{i} \quad \forall i \in \{1,2,3,4,5,6,7,8,9,10\}$$

9. Regression Deviation Constraints:

The regression estimates for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can deviate by 2%, 1%, 5%, and 10%, respectively. These constraints can be formulated as follows:

$$-0.02 \cdot x_{4} \leq \hat{x}_{4} - x_{4} \leq 0.02 \cdot x_{4}$$
$$-0.01 \cdot x_{7} \leq \hat{x}_{7} - x_{7} \leq 0.01 \cdot x_{7}$$
$$-0.05 \cdot x_{9} \leq \hat{x}_{9} - x_{9} \leq 0.05 \cdot x_{9}$$
$$-0.10 \cdot x_{10} \leq \hat{x}_{10} - x_{10} \leq 0.10 \cdot x_{10}$$

where $\hat{x}_{4}$, $\hat{x}_{7}$, $\hat{x}_{9}$, and $\hat{x}_{10}$ are the estimated values of alkylate yield, motor octane number, acid dilution factor, and F-4 performance number, respectively.

This completes the formulation of the mathematical optimization model.

In [7]:
print(response_1_text)

To formulate a mathematical optimization model for this problem, we will define the parameters, decision variables, objective function, and constraints as follows:

Parameters:

* $C_{AF}$: Cost per barrel of olefin feed
* $C_{IR}$: Cost per barrel of isobutane recycle
* $C_{AAR}$: Cost per thousand pounds of acid addition rate
* $C_{IM}$: Cost per barrel of isobutane makeup
* $P_{AY}$: Price per barrel of alkylate yield
* $P_{MON}$: Price per unit of motor octane number

Decision Variables:

* $x_{1}$: Olefin feed (barrels per day)
* $x_{2}$: Isobutane recycle (barrels per day)
* $x_{3}$: Acid addition rate (thousands of pounds per day)
* $x_{4}$: Alkylate yield (barrels per day)
* $x_{5}$: Isobutane makeup (barrels per day)
* $x_{6}$: Acid strength (weight %)
* $x_{7}$: Motor octane number
* $x_{8}$: External isobutane-to-olefin ratio
* $x_{9}$: Acid dilution factor
* $x_{10}$: F-4 performance number

Objective Function:

The objective is to maximize the daily profit, which is define

## Step 2 - Generate the Pyomo Code

In [8]:
messages_2 = [
    ChatMessage(role="system", content=SYSTEM_PROMPT_2),
    ChatMessage(role="user", content=response_1_text)
]

In [9]:
response_2 = client.chat(
    model=MODEL_ID,
    messages=messages_2,
    random_seed=MODEL_SEED,
    temperature=MODEL_TEMPERATURE
)

response_2_text = response_2.choices[0].message.content

In [10]:
Markdown(response_2_text)

Here is a Python Pyomo code for the given optimization problem. Please note that I will use sample data for the parameters and bounds. You can replace them with actual values as needed.

```python
from pyomo.environ import *

# Sample data (replace with actual values)
C_AF = 100  # Cost per barrel of olefin feed
C_IR = 50  # Cost per barrel of isobutane recycle
C_AAR = 20  # Cost per thousand pounds of acid addition rate
C_IM = 80  # Cost per barrel of isobutane makeup
P_AY = 150  # Price per barrel of alkylate yield
P_MON = 10  # Price per unit of motor octane number

# Define the model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(within=NonNegativeReals, bounds=(10, 50))  # Olefin feed
model.x2 = Var(within=NonNegativeReals, bounds=(10, 50))  # Isobutane recycle
model.x3 = Var(within=NonNegativeReals, bounds=(10, 50))  # Acid addition rate
model.x4 = Var(within=NonNegativeReals, bounds=(10, 50))  # Alkylate yield
model.x5 = Var(within=NonNegativeReals, bounds=(10, 50))  # Isobutane makeup
model.x6 = Var(within=NonNegativeReals, bounds=(10, 50))  # Acid strength
model.x7 = Var(within=NonNegativeReals, bounds=(10, 50))  # Motor octane number
model.x8 = Var(within=NonNegativeReals, bounds=(10, 50))  # External isobutane-to-olefin ratio
model.x9 = Var(within=NonNegativeReals, bounds=(10, 50))  # Acid dilution factor
model.x10 = Var(within=NonNegativeReals, bounds=(10, 50))  # F-4 performance number

# Objective function
def obj_rule(model):
    return P_AY * model.x4 * model.x7 - (C_AF * model.x1 + C_IR * model.x2 + C_AAR * model.x3 + C_IM * model.x5)

model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
model.c1 = Constraint(expr=model.x4 == model.x1 * (1.12 + 0.13167 * model.x8 - 0.00667 * model.x8**2))
model.c2 = Constraint(expr=model.x7 == 86.35 + 1.098 * model.x8 - 0.038 * model.x8**2 + 0.325 * (model.x6 - 89))
model.c3 = Constraint(expr=model.x9 == 35.82 - 0.222 * model.x10)
model.c4 = Constraint(expr=model.x10 == -133 + 3 * model.x7)
model.c5 = Constraint(expr=model.x8 == (model.x2 + model.x5) / model.x1)
model.c6 = Constraint(expr=model.x6 == (93000 * model.x3) / (model.x3 * model.x9 + 1000 * model.x3))
model.c7 = Constraint(expr=1.22 * model.x4 == model.x1 + model.x5)

# Regression deviation constraints
model.c8 = Constraint(expr=-0.02 * model.x4 <= 1.12 * model.x1 * model.x8 + 0.13167 * model.x1 * model.x8**2 - 0.00667 * model.x1 * model.x8**3 - model.x4)
model.c9 = Constraint(expr=-0.01 * model.x7 <= 86.35 * model.x8 + 1.098 * model.x8**2 - 0.038 * model.x8**3 + 0.325 * model.x6 - 0.325 * 89 * model.x8 - model.x7)
model.c10 = Constraint(expr=-0.05 * model.x9 <= 35.82 * model.x10 - 0.222 * model.x10**2 - model.x9)
model.c11 = Constraint(expr=-0.10 * model.x10 <= -133 * model.x7 + 3 * model.x7**2 - model.x10)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Optimal solution found:")
print(f"x1: {value(model.x1)}")
print(f"x2: {value(model.x2)}")
print(f"x3: {value(model.x3)}")
print(f"x4: {value(model.x4)}")
print(f"x5: {value(model.x5)}")
print(f"x6: {value(model.x6)}")
print(f"x7: {value(model.x7)}")
print(f"x8: {value(model.x8)}")
print(f"x9: {value(model.x9)}")
print(f"x10: {value(model.x10)}")
```

In [11]:
print(response_2_text)

Here is a Python Pyomo code for the given optimization problem. Please note that I will use sample data for the parameters and bounds. You can replace them with actual values as needed.

```python
from pyomo.environ import *

# Sample data (replace with actual values)
C_AF = 100  # Cost per barrel of olefin feed
C_IR = 50  # Cost per barrel of isobutane recycle
C_AAR = 20  # Cost per thousand pounds of acid addition rate
C_IM = 80  # Cost per barrel of isobutane makeup
P_AY = 150  # Price per barrel of alkylate yield
P_MON = 10  # Price per unit of motor octane number

# Define the model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(within=NonNegativeReals, bounds=(10, 50))  # Olefin feed
model.x2 = Var(within=NonNegativeReals, bounds=(10, 50))  # Isobutane recycle
model.x3 = Var(within=NonNegativeReals, bounds=(10, 50))  # Acid addition rate
model.x4 = Var(within=NonNegativeReals, bounds=(10, 50))  # Alkylate yield
model.x5 = Var(within=NonNegativeReals, bounds=(

### Code Executability

In [12]:
from pyomo.environ import *

# Sample data (replace with actual values)
C_AF = 100  # Cost per barrel of olefin feed
C_IR = 50  # Cost per barrel of isobutane recycle
C_AAR = 20  # Cost per thousand pounds of acid addition rate
C_IM = 80  # Cost per barrel of isobutane makeup
P_AY = 150  # Price per barrel of alkylate yield
P_MON = 10  # Price per unit of motor octane number

# Define the model
model = ConcreteModel()

# Define decision variables
model.x1 = Var(within=NonNegativeReals, bounds=(10, 50))  # Olefin feed
model.x2 = Var(within=NonNegativeReals, bounds=(10, 50))  # Isobutane recycle
model.x3 = Var(within=NonNegativeReals, bounds=(10, 50))  # Acid addition rate
model.x4 = Var(within=NonNegativeReals, bounds=(10, 50))  # Alkylate yield
model.x5 = Var(within=NonNegativeReals, bounds=(10, 50))  # Isobutane makeup
model.x6 = Var(within=NonNegativeReals, bounds=(10, 50))  # Acid strength
model.x7 = Var(within=NonNegativeReals, bounds=(10, 50))  # Motor octane number
model.x8 = Var(within=NonNegativeReals, bounds=(10, 50))  # External isobutane-to-olefin ratio
model.x9 = Var(within=NonNegativeReals, bounds=(10, 50))  # Acid dilution factor
model.x10 = Var(within=NonNegativeReals, bounds=(10, 50))  # F-4 performance number

# Objective function
def obj_rule(model):
    return P_AY * model.x4 * model.x7 - (C_AF * model.x1 + C_IR * model.x2 + C_AAR * model.x3 + C_IM * model.x5)

model.obj = Objective(rule=obj_rule, sense=maximize)

# Constraints
model.c1 = Constraint(expr=model.x4 == model.x1 * (1.12 + 0.13167 * model.x8 - 0.00667 * model.x8**2))
model.c2 = Constraint(expr=model.x7 == 86.35 + 1.098 * model.x8 - 0.038 * model.x8**2 + 0.325 * (model.x6 - 89))
model.c3 = Constraint(expr=model.x9 == 35.82 - 0.222 * model.x10)
model.c4 = Constraint(expr=model.x10 == -133 + 3 * model.x7)
model.c5 = Constraint(expr=model.x8 == (model.x2 + model.x5) / model.x1)
model.c6 = Constraint(expr=model.x6 == (93000 * model.x3) / (model.x3 * model.x9 + 1000 * model.x3))
model.c7 = Constraint(expr=1.22 * model.x4 == model.x1 + model.x5)

# Regression deviation constraints
model.c8 = Constraint(expr=-0.02 * model.x4 <= 1.12 * model.x1 * model.x8 + 0.13167 * model.x1 * model.x8**2 - 0.00667 * model.x1 * model.x8**3 - model.x4)
model.c9 = Constraint(expr=-0.01 * model.x7 <= 86.35 * model.x8 + 1.098 * model.x8**2 - 0.038 * model.x8**3 + 0.325 * model.x6 - 0.325 * 89 * model.x8 - model.x7)
model.c10 = Constraint(expr=-0.05 * model.x9 <= 35.82 * model.x10 - 0.222 * model.x10**2 - model.x9)
model.c11 = Constraint(expr=-0.10 * model.x10 <= -133 * model.x7 + 3 * model.x7**2 - model.x10)

# Solve the model
solver = SolverFactory('ipopt')
results = solver.solve(model)

# Print the results
print("Optimal solution found:")
print(f"x1: {value(model.x1)}")
print(f"x2: {value(model.x2)}")
print(f"x3: {value(model.x3)}")
print(f"x4: {value(model.x4)}")
print(f"x5: {value(model.x5)}")
print(f"x6: {value(model.x6)}")
print(f"x7: {value(model.x7)}")
print(f"x8: {value(model.x8)}")
print(f"x9: {value(model.x9)}")
print(f"x10: {value(model.x10)}")

  - termination condition: infeasible
  - message from solver: Ipopt 3.12.13\x3a Converged to a locally infeasible point. Problem may be infeasible.


Optimal solution found:
x1: 10.0
x2: 50.0
x3: 10.19630876617798
x4: 17.696999824763434
x5: 11.590339886209753
x6: 50.0
x7: 10.0
x8: 10.0
x9: 33.60000002220005
x10: 10.0


### Solution Correctness